In [24]:
import sys
import os
import cv2
import numpy as np
from PIL import Image
from tqdm import tqdm_notebook as tqdm
 

In [25]:
import pyautogui

In [3]:
directory = "data/train/1"
imagecount = 1000

In [4]:
os.makedirs(directory, exist_ok=True)

In [5]:
video = cv2.VideoCapture(0)

[ WARN:0@2.154] global /private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_11nitadzeg/croot/opencv-suite_1691620374638/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created
2024-04-25 17:22:25.578 python[68410:19282747] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


Code referred and used from https://medium.com/swlh/hand-gestures-using-webcam-and-cnn-convoluted-neural-network-b02c47b3d5ab

In [70]:
filename = len(os.listdir(directory))
count = 0
pbar = tqdm(total = imagecount+1)
while True and count < imagecount:
    filename += 1
    count += 1
    _, frame = video.read()
    kernel = np.ones((3,3),np.uint8)
     
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)# define range of skin color in HSV
    lower_skin = np.array([0,20,70], dtype=np.uint8)
    upper_skin = np.array([255,255,255], dtype=np.uint8)#extract skin colur image
    mask = cv2.inRange(hsv, lower_skin, upper_skin)#extrapolate the hand to fill dark spots within
    mask = cv2.dilate(mask,kernel,iterations = 4)#blur the image
    mask = cv2.GaussianBlur(mask,(5,5),100)
    path = directory+"//"+str(filename)+".jpg"
    cv2.imwrite(path, mask)
    cv2.imshow("Capturing", mask)
    key=cv2.waitKey(1)
    if key == ord('q'):
        break
pbar.update(1)
pbar.close()
video.release()
cv2.destroyAllWindows()

/var/folders/2c/9kbb32qx5yg0m_74lsfqyr6c0000gn/T/ipykernel_55859/2873970826.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  pbar = tqdm(total = imagecount+1)


  0%|          | 0/1001 [00:00<?, ?it/s]

In [7]:
#pbar.update(1)
#pbar.close()
video.release()
cv2.destroyAllWindows()


In [26]:
from keras import models
from keras import layers
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator

In [27]:
Imagesize=128
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(Imagesize,Imagesize,3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(2, 2))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(2, 2))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(2, 2))
model.add(layers.Flatten())
model.add(layers.Dropout(0.25))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(4, activation='softmax'))

In [28]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 63, 63, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 30, 30, 64)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 28, 28, 128)       73856     
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 14, 14, 128)     

In [29]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [30]:
data_train = ImageDataGenerator(rescale=1./255,rotation_range=40,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
data_valid = ImageDataGenerator(rescale=1.255)
data_test = ImageDataGenerator(rescale=1.255)

In [31]:
train_gen = data_train.flow_from_directory('data/train',target_size=(Imagesize,Imagesize),batch_size=108, class_mode='categorical')
valid_gen = data_valid.flow_from_directory('data/valid', target_size=(Imagesize,Imagesize), batch_size=108, class_mode='categorical')
test_gen = data_test.flow_from_directory('data/test', target_size=(Imagesize,Imagesize), batch_size=108, class_mode='categorical')

Found 2404 images belonging to 4 classes.
Found 291 images belonging to 4 classes.
Found 152 images belonging to 4 classes.


In [32]:
model.fit_generator(train_gen, epochs=10, steps_per_epoch=47, validation_data=valid_gen, validation_steps=7, workers=4)

/var/folders/2c/9kbb32qx5yg0m_74lsfqyr6c0000gn/T/ipykernel_61228/1167385744.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_gen, epochs=10, steps_per_epoch=47, validation_data=valid_gen, validation_steps=7, workers=4)


Epoch 1/10


2024-05-03 13:30:29.453211: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


47/47 [==============================] - ETA: 0s - loss: 0.8124 - accuracy: 0.6633

2024-05-03 13:31:03.861503: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


47/47 [==============================] - 36s 750ms/step - loss: 0.8124 - accuracy: 0.6633 - val_loss: 28.8978 - val_accuracy: 0.8870
Epoch 2/10
47/47 [==============================] - 38s 775ms/step - loss: 0.3437 - accuracy: 0.8594 - val_loss: 6.3234 - val_accuracy: 0.9464
Epoch 3/10
47/47 [==============================] - 39s 819ms/step - loss: 0.2494 - accuracy: 0.8993 - val_loss: 81.2012 - val_accuracy: 0.8174
Epoch 4/10
47/47 [==============================] - 40s 833ms/step - loss: 0.1978 - accuracy: 0.9258 - val_loss: 13.5161 - val_accuracy: 0.9551
Epoch 5/10
47/47 [==============================] - 41s 849ms/step - loss: 0.1545 - accuracy: 0.9426 - val_loss: 7.5909 - val_accuracy: 0.9609
Epoch 6/10
47/47 [==============================] - 41s 861ms/step - loss: 0.1387 - accuracy: 0.9518 - val_loss: 7.6259 - val_accuracy: 0.9739
Epoch 7/10
47/47 [==============================] - 43s 893ms/step - loss: 0.1296 - accuracy: 0.9467 - val_loss: 0.5464 - val_accuracy: 0.9899
Epoch 8

In [11]:
model.save("model1")

2024-04-30 21:53:31.848440: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,9216]
	 [[{{node inputs}}]]
2024-04-30 21:53:31.983938: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,9216]
	 [[{{node inputs}}]]


INFO:tensorflow:Assets written to: model1/assets


INFO:tensorflow:Assets written to: model1/assets


In [33]:
class_names = test_gen.labels
class_names 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
      dtype=int32)

In [35]:
predictions = model.predict(test_gen)

predicted_labels = predictions.argmax(axis=1)



2024-05-03 13:38:20.266996: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


2/2 [==============================] - 0s 72ms/step


In [36]:
loss, accuracy = model.evaluate(test_gen)


2024-05-03 13:38:20.840246: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


2/2 [==============================] - 0s 75ms/step - loss: 0.0000e+00 - accuracy: 1.0000


In [23]:
accuracy

1.0

In [14]:
model=models.load_model('model1')


In [66]:
screenWidth, screenHeight = pyautogui.size()

In [22]:
video = cv2.VideoCapture(0)
label_mapping = {0: "01_palm", 1: '02_l', 2: '03_fist', 3: '04_fist_moved',4:'05_thumb',5:'06_index',6:'07_ok',7:'08_palm_moved',8:'09_c',9:'10_down'}



[ WARN:0@878.398] global /private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_11nitadzeg/croot/opencv-suite_1691620374638/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


In [23]:
while True:
    _, frame = video.read()
    kernel = np.ones((3,3),np.uint8)
     
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)# define range of skin color in HSV
    lower_skin = np.array([0,20,70], dtype=np.uint8)
    upper_skin = np.array([20,255,255], dtype=np.uint8)#extract skin colur image
    mask = cv2.inRange(hsv, lower_skin, upper_skin)#extrapolate the hand to fill dark spots within
    mask = cv2.dilate(mask,kernel,iterations = 4)#blur the image
    mask = cv2.GaussianBlur(mask,(5,5),100)
    mask = cv2.resize(mask,(128,128))
    img_array = np.array(mask)
    #print(img_array.shape)# Changing dimension from 128x128 to 128x128x3
    img_array = np.stack((img_array,)*3, axis=-1)
    #Our keras model used a 4D tensor, (images x height x width x channel)
    #So changing dimension 128x128x3 into 1x128x128x3 
    img_array_ex = np.expand_dims(img_array, axis=0)
    print(img_array_ex.shape)
    #Calling the predict method on model to predict gesture in the frame
    prediction = model.predict(img_array_ex)
    print(prediction)
    prediction = np.argmax(prediction)
    cv2.putText(frame, f"Prediction: {label_mapping[prediction]}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
#     if (prediction == 0):
            
#         pyautogui.press('up')
#     if (prediction == 3):
#         pyautogui.press('down') 
    cv2.imshow("Capturing", frame)
    key=cv2.waitKey(1)
    if key == ord('q'):
        break
video.release()
cv2.destroyAllWindows()

(1, 128, 128, 3)
1/1 [==============================] - 0s 116ms/step
[[1. 0. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 17ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 14ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 14ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 14ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 15ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 19ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 19ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 12ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/

1/1 [==============================] - 0s 12ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 12ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 14ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.

1/1 [==============================] - 0s 12ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 12ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 24ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 24ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 20ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 14ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 12ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 15ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.

1/1 [==============================] - 0s 13ms/step
[[0. 0. 0. 1.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 0. 0. 1.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 0. 0. 1.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 12ms/step
[[0. 0. 0. 1.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 0. 0. 1.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 0. 0. 1.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 0. 0. 1.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 0. 0. 1.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 0. 0. 1.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 0. 0. 1.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 0. 0. 1.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 0. 0. 1.

1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 14ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 12ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 14ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 14ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 12ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 12ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.

...

1/1 [==============================] - 0s 12ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 12ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 14ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 12ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 12ms/step
[[0. 1. 0. 0.

1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 12ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 12ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 12ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 14ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.

1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.

1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 14ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 12ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.

1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 12ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 12ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 12ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 14ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.]]
(1, 128, 128, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 1. 0. 0.